In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date, datetime, timedelta
import data_preperation as prep

In [3]:
#get measurement data and forecast data: 
df0=prep.prepare_measurement()
forecast = prep.prepare_forecast()

/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/Users/cynthiazeng/safi/data_preperation.py:65: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  data0 = pd.concat([df0, df1, df2, df3, df4, df5, df6, df7, df8, df9], ignore_index=True)


read csv semester csv files from 2015s2 to 2020s1


/Users/cynthiazeng/safi/data_preperation.py:82: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  df[c]= pd.to_numeric(df[c], errors= 'coerce')


smooth wind direction
generate seasonality categorical feature
generate day/night categorical feature


/Users/cynthiazeng/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3325: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


reading forecast data
smooth wind direction


In [4]:
# data_merge, data, forecast = prepare_data_with_forecast(data, keep_only_last=False)
def get_past_n_steps(df, steps_in):
    #rename column to most remote data
    df_out = df.copy().add_suffix('_t-'+str(steps_in))
    #t-i remote data
    for i in range(1, steps_in+1):
        df_temp = df.copy().add_suffix('_t-'+str(steps_in-i)) #rename column
        df_temp= df_temp.shift(periods=-i, axis=0) #shift down i row
        df_out=df_out.join(df_temp, how = 'inner')#join
    #shift index to present time (+steps_in)
    df_out['datetime_now']=df_out.index.to_series()+timedelta(hours=steps_in)
    df_out.set_index(pd.DatetimeIndex(df_out['datetime_now']), inplace=True)
    return df_out

In [6]:
def join_forecast(df, forecast, predict): 
    
    #crop out forecast if forecast period is less than prediction period
    forecast = forecast.loc[forecast['f_period']>= predict]
    forecast = prep.keep_last_forecast(forecast) 
    forecast = forecast.add_suffix('_forecast')
#     forecast.drop(['f_date','p_date'], axis=1, inplace=True)
    
    #join forecast with data 
    df['datetime_predict']=df['datetime_now']+timedelta(hours=predict)
    df.set_index(df['datetime_predict'], inplace=True)
    
    df_out = df.join(forecast, how= 'left')
    return df_out 

df = get_past_n_steps(df0, steps_in=3)
predict = 2
df = join_forecast(df, forecast, predict)
df.head()


,cos_wind_dir_t-3,sin_wind_dir_t-3,speed_t-3,temp_t-3,radiation_t-3,precip_t-3,d_speed_min_t-3,d_speed_max_t-3,d_temp_min_t-3,d_temp_max_t-3,...,day_t-0,night_t-0,datetime_now,datetime_predict,speed_forecast,temp_forecast,rad_forecast,precip_forecast,cos_wind_dir_forecast,sin_wind_dir_forecast
datetime_predict,,,,,,,,,,,,,,,,,,,,,
2015-07-01 05:00:00,0.850439,0.211122,2.225,20.725,0.0,0.0,0.85,8.85,20.325,24.075,...,0.0,1.0,2015-07-01 03:00:00,2015-07-01 05:00:00,1.080946,19.229883,0.000000,0.0172119997441769,0.980986,0.194076
2015-07-01 06:00:00,0.847033,-0.517158,2.900,20.500,0.0,0.0,0.85,8.85,20.325,24.075,...,0.0,1.0,2015-07-01 04:00:00,2015-07-01 06:00:00,0.981995,19.389978,0.000000,0.0266730021685362,0.981373,0.192114
2015-07-01 07:00:00,0.929234,-0.082164,1.600,20.675,0.0,0.0,0.85,8.85,20.325,24.075,...,0.0,1.0,2015-07-01 05:00:00,2015-07-01 07:00:00,0.865024,19.629999,142.224380,0.0257569998502731,0.990156,0.139970
2015-07-01 08:00:00,0.812413,0.389547,2.175,20.850,0.0,0.0,0.85,8.85,20.325,24.075,...,0.0,1.0,2015-07-01 06:00:00,2015-07-01 08:00:00,0.851891,20.269983,342.239685,0.0196529999375343,0.998515,0.054470
2015-07-01 09:00:00,0.472097,0.850407,2.625,20.800,0.0,0.0,0.85,8.85,20.325,24.075,...,0.0,1.0,2015-07-01 07:00:00,2015-07-01 09:00:00,0.995341,21.350031,526.759399,0,0.984512,-0.175315


In [ ]:
#define y according to x_df and prediction period 
def get_y (x_df, predict):
    y_df = x_df['scenario_num_t-0'] 
    #shift df by prediction period 
    y_df = y_df[predict:]
    x_df = x_df[:-predict]
    return x_df, y_df 

# x_df, y_df = get_y(df, predict)


#change x,y to array like 
def df_to_array(X_df, Y_df):
    #drop timestamp columns
    X_df.drop(['datetime_now','datetime_predict'], axis=1, inplace=True)
    #forward fill: use last forecast
    X_df.fillna(method='ffill', inplace=True)
    Y_df.fillna(method='ffill', inplace=True) 
    #dropna after filling nan 
    X_df.dropna(axis=1, inplace=True)
    X_df.dropna(axis=1, inplace=True)
    X = X_df.values
    Y = Y_df.values 
    return X, Y
# X,Y = df_to_array(X_df, Y_df)

def prepare_x_y(past_period, predict_period, historical_df, forecast_df):
    # concatenate past_n_steps data 
    df=get_past_n_steps(historical_df, past_period)
    # add forecast according to prediction period 
    x_df = join_forecast(df, forecast_df, predict_period)
    # define y according to x 
    x_df, y_df = get_y(x_df, predict_period)
    # change dataframe to array
    x, y = df_to_array(x_df, y_df) 
    return x, y
    
# x, y = prepare_x_y(12, 12, data, forecast)